# STA4020 Final Project
### Objective:
- explore the practical applications of asset allocation methodologies in constructing and evaluating investment portfolios. 

### Aims:
- The project aims to help you apply the following key concepts:
    - mean-variance optimization (MVO)
    - equal-weighted portfolios (1/N)
    - the BlackLitterman model for incorporating views. 

</br>
The focus is on analyzing the performance of portfolios both in-sample and out-of-sample, under the given constraints.

### Step 1: Data Collection

This reports uses the following 5 stocks in the chosen portfolio. The original data is a 5-year dataset counting backwards from December 2024. The data contains daily open-close prices of the stock.

**Portfolio Construction:**
1) Visa Inc. (V)
2) Mastercard Inc. (MA)
3) American Express Company (AXP)
4) Palantir Technologies Inc (PLTR)
5) NIO Inc. (NIO)

In [ ]:
import pandas as pd

# Define the ticker symbols
tickers = ['pltr', 'nio', 'v', 'ma', 'axp']

# Initialize a dictionary to store the data
stock_data = {}

# Loop through each ticker symbol and read the corresponding CSV file
for ticker in tickers:
    file_name = f"{ticker}_quotes.csv"  # Construct the file name
    stock_data[ticker] = pd.read_csv(file_name)  # Read the CSV file and store in dictionary

# Accessing individual stock data
pltr_data = stock_data['pltr']
nio_data = stock_data['nio']
v_data = stock_data['v']
ma_data = stock_data['ma']
axp_data = stock_data['axp']

**Data preprocessing steps:**
1) Check (and remove) any missing values
2) Truncate the 5 year history into a 2 year and 3 months history
    - `3 months`: Sept 2024 - Nov 2024
    - `2 years`: Aug 2022 - Aug 2024
3) Convert String values to Float (due to the `$` character)
4) Add `Returns` and `% Returns` column for the processed data
5) Partition the datasets into In-Sample and Out-Sample
    - `in_sample`: 2 years of previous data
    - `out_sample`: 3 months of latest data

**Formulae used for computing `Returns` and `% Returns`:**
1. **Daily Return (in dollars)**:
   $$\text{Daily Return} = (P_{\text{closing}} - P_{\text{opening}}) $$
   where:
   - $ P_{\text{closing}} $ = closing price
   - $ P_{\text{opening}} $ = opening price

2. **Percentage Return**:
   $$\text{Percentage Return} = \left( \frac{P_{\text{closing}} - P_{\text{opening}}}{P_{\text{opening}}} \right) \times 100$$


*note that in our dataframes, the `% Returns` columns are kept as a value between 0 & 1

In [ ]:
# Check for null values in each stock data
null_checks = {
    "Palantir": pltr_data.isnull().sum().sum(),
    "NIO": nio_data.isnull().sum().sum(),
    "Visa": v_data.isnull().sum().sum(),
    "Mastercard": ma_data.isnull().sum().sum(),
    "American Express": axp_data.isnull().sum().sum()
}

all_datasets_valid = True

for company, null_count in null_checks.items():
    if null_count > 0:
        print(f"{company} DataFrame has {null_count} null values.")
        all_datasets_valid = False

if all_datasets_valid:
    print("All datasets do not have null values.")

All datasets do not have null values.


Here we define some preprocessing functions

In [34]:
# Define function to convert the 'Date' column to datetime format
def convert_to_datetime(data):
    data['Date'] = pd.to_datetime(data['Date'])
    return data

# Define function to keep all rows within a specified date range and reindex the dataframe
def filter_by_date(data, start_date, end_date):
    data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
    data = data.reset_index(drop=True)
    return data

# Define function to remove '$' character from the selected columns and convert the remaining values to float
def clean_currency(x):
    if isinstance(x, str):
        return x.replace('$', '').replace(',', '')
    return x

# Define the columns to clean
columns_to_clean = ['Close/Last', 'Open', 'High', 'Low']

# Define function to calculate Returns and % Returns
def calculate_returns(data):
    data['Returns'] = data['Open'].astype(float) - data['Close/Last'].astype(float)
    data['% Returns'] = data['Returns'] / data['Open'].astype(float)
    return data

Then we apply these functions onto our datasets

In [35]:
# Apply the functions to each stock data and update the dictionary
for ticker in tickers:
    stock_data[ticker] = convert_to_datetime(stock_data[ticker])
    stock_data[ticker] = filter_by_date(stock_data[ticker], '2022-08-01', '2024-11-30')
    stock_data[ticker][columns_to_clean] = stock_data[ticker][columns_to_clean].map(clean_currency)
    stock_data[ticker] = calculate_returns(stock_data[ticker])

# Accessing individual stock data
pltr_data = stock_data['pltr']
nio_data = stock_data['nio']
v_data = stock_data['v']
ma_data = stock_data['ma']
axp_data = stock_data['axp']

Now we partition our datasets into `in_sample` and `out_sample`

In [37]:
# Create new dictionary to store each partition of the original data
stock_data_in_sample = {}
stock_data_out_sample = {}

# Loop through each ticker symbol and split the data into in-sample and out-of-sample
for ticker in tickers:
    stock_data_in_sample[ticker] = stock_data[ticker].loc[stock_data[ticker]['Date'] <= '2024-08-31'].reset_index(drop=True)
    stock_data_out_sample[ticker] = stock_data[ticker].loc[stock_data[ticker]['Date'] > '2024-08-31'].reset_index(drop=True)

# Accessing individual stock data
pltr_data_in_sample = stock_data_in_sample['pltr']
nio_data_in_sample = stock_data_in_sample['nio']
v_data_in_sample = stock_data_in_sample['v']
ma_data_in_sample = stock_data_in_sample['ma']
axp_data_in_sample = stock_data_in_sample['axp']

pltr_data_out_sample = stock_data_out_sample['pltr']
nio_data_out_sample = stock_data_out_sample['nio']
v_data_out_sample = stock_data_out_sample['v']
ma_data_out_sample = stock_data_out_sample['ma']
axp_data_out_sample = stock_data_out_sample['axp']


In [38]:
v_data_in_sample.head()

,Date,Close/Last,Volume,Open,High,Low,Returns,% Returns
0,2024-08-30,276.37,6296089,275.00,276.99,274.26,-1.37,-0.004982
1,2024-08-29,274.32,5565085,270.98,275.99,270.45,-3.34,-0.012326
2,2024-08-28,269.19,5455098,271.16,271.98,267.67,1.97,0.007265
3,2024-08-27,270.72,6594067,268.52,270.95,267.89,-2.20,-0.008193
4,2024-08-26,268.21,4612181,267.72,270.37,267.67,-0.49,-0.001830


### Step 2: MVE Portfolio Construction

### Step 3: Black Litterman Model

### Step 4: Out-of-sample Portfolio Evaluation